In [11]:
import json
import numpy as np
import os.path as osp
import codex
from codex import simulation as codex_sim
from codex import config as codex_config
from codex import io as codex_io
from flowdec.nb import utils as nb_utils
from flowdec import data as fd_data

In [2]:
src_config = codex_config.load_example_config('ex1')

In [34]:
dict(x={1:2})

{'x': {1: 2}}

In [46]:
x = dict(x={1:3}, y={3:4, 5:6})
eval(str(x)) == x

True

In [26]:
img = fd_data.load_celegans_channel('FITC').data
img = img[slice(0, 160, 25)]
img.shape

(5, 712, 672)

In [27]:
def split(img, nrows, ncols):
    nz, nh, nw = img.shape
    ih = nh // nrows
    iw = nw // ncols
    res = []
    for i in range(nrows):
        row = []
        for j in range(ncols):
            start_row = ih * i
            stop_row = ih * (i+1)
            start_col = iw * j
            stop_col = iw * (j + 1)
            row.append(img[:, start_row:stop_row, start_col:stop_col])
        res.append(row)
    # Return result as (rows, cols, nz, nh, nw)
    return np.array(res)

img_split = split(img, 3, 2)
img_split.shape

(3, 2, 5, 237, 336)

In [28]:
tile, config, info = codex_sim.experiment_from_img(img_split[0, 0], src_config=src_config, nch=2, ncyc=2)

In [29]:
def save_simulation_experiment(data_dir, img, region=0, nch=2, ncyc=2):
    nrows, ncols, nz = img.shape[0], img.shape[1], img.shape[2]
    first_config = None
    for irow in range(nrows):
        for icol in range(ncols):
            itile = irow * ncols + icol
            # Tile will be 5D stack shaped like (cycles, z, channel, height, width)
            tile, config, info = codex_sim.experiment_from_img(img[irow, icol], src_config=src_config, nch=nch, ncyc=ncyc)
            if first_config is None:
                first_config = config
            for icyc in range(ncyc):
                for ich in range(nch):
                    for iz in range(nz):
                        # Select height x width image
                        img_patch = tile[icyc, iz, ich]
                        path = codex_io.get_raw_img_path(region, itile, icyc, ich, iz)
                        print('Saving tile to path {}'.format(path))
                        codex_io.save_tile(osp.join(data_dir, path), img_patch)
                        
    conf_data = dict(first_config._conf)
    conf_data['region_width'] = ncols
    conf_data['region_height'] = nrows
    all_channel_names = conf_data['all_channel_names']
    del conf_data['all_channel_names']
    
    conf_path = osp.join(data_dir, 'Experiment.json')
    with open(conf_path, 'w') as fd:
        print('Saving experiment config to path {}'.format(conf_path))
        json.dump(conf_data, fd)
        
    conf_path = osp.join(data_dir, 'channelNames.txt')
    with open(conf_path, 'w') as fd:
        print('Saving channel names to path {}'.format(conf_path))
        fd.write('\n'.join(all_channel_names))

In [31]:
data_dir = osp.expanduser('~/data/research/hammer/codex/simulations/sim-exp-01')
save_simulation_experiment(data_dir, img_split)

Saving tile to path Cyc1_reg1/1_00001_Z001_CH1.tif
Saving tile to path Cyc1_reg1/1_00001_Z002_CH1.tif
Saving tile to path Cyc1_reg1/1_00001_Z003_CH1.tif
Saving tile to path Cyc1_reg1/1_00001_Z004_CH1.tif
Saving tile to path Cyc1_reg1/1_00001_Z005_CH1.tif
Saving tile to path Cyc1_reg1/1_00001_Z001_CH2.tif
Saving tile to path Cyc1_reg1/1_00001_Z002_CH2.tif
Saving tile to path Cyc1_reg1/1_00001_Z003_CH2.tif
Saving tile to path Cyc1_reg1/1_00001_Z004_CH2.tif
Saving tile to path Cyc1_reg1/1_00001_Z005_CH2.tif
Saving tile to path Cyc2_reg1/1_00001_Z001_CH1.tif
Saving tile to path Cyc2_reg1/1_00001_Z002_CH1.tif
Saving tile to path Cyc2_reg1/1_00001_Z003_CH1.tif
Saving tile to path Cyc2_reg1/1_00001_Z004_CH1.tif
Saving tile to path Cyc2_reg1/1_00001_Z005_CH1.tif
Saving tile to path Cyc2_reg1/1_00001_Z001_CH2.tif
Saving tile to path Cyc2_reg1/1_00001_Z002_CH2.tif
Saving tile to path Cyc2_reg1/1_00001_Z003_CH2.tif
Saving tile to path Cyc2_reg1/1_00001_Z004_CH2.tif
Saving tile to path Cyc2_reg1/1